# 8-4. 텍스트 분류 실습 - 20 뉴스그룹 분류

	• what to do? 20 뉴스그룹 데이터 세트를 이용해 텍스트 분류를 적용하기
	• 텍스트 분류란? 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 학습 모델을 이용해 다른 문서의 분류를 예측
    사이킷런의 fetch_20newsgroups() : 예제 데이터 제공
	• 텍스트 정규화 > 텍스트의 피처 벡터화 (희소 행렬 형태로 > 로지스틱 회귀로 분류 수행) > 머신러닝 알고리즘 적용

## 1. 텍스트 정규화

In [5]:
#fetch_20newsgroups()로 인터넷에서 로컬로 데이터 내려받기
from sklearn.datasets import fetch_20newsgroups
#로컬 데이터 경로
data_home="/Users/bluecloud/Documents/대학/유런/데이터셋/20news-bydate-test"
# 데이터셋 로드
news_data = fetch_20newsgroups(data_home=data_home, subset='all', random_state=156)

In [13]:
#키값 확인
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [15]:
#target 클래스 구성 확인
import pandas as pd

print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index)
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 <bound method Series.sort_index of 10    999
15    997
8     996
9     994
11    991
13    990
7     990
5     988
14    987
2     985
12    984
3     982
6     975
1     973
4     963
17    940
16    910
0     799
18    775
19    628
Name: count, dtype: int64>
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [17]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

>> 텍스트 데이터 확인 결과 : 기사내용+제목+작성자+소속+이메일 등 정보 다양함
>> 내용 빼고 제거함.(나머지 정보는 target 클래스 값과 유사한 데이터 갖고 있는 경우가 많아서)

then how? remove 파라미터로 header와 footer 등 제거 가능
also, subset 파라미터로 학습 데이터 세트와 테스트 데이터 세트 분리해서 내려받기

In [3]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news=fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),random_state=156)
X_train = train_news.data
y_train = train_news.target

# subset='test'으로 테스트 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
test_news=fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))

학습 데이터 크기 11314 , 테스트 데이터 크기 7532


## 2. 피처 벡터화 변환과 머신러닝 모델 학/예/평

CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화하기
- 테스트 데이터 >> 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체로 변환해야함
- cnt_vect.transform()해야 한다.(fit_transform() 적용 X > 테스트 데이터 기반으로 작동되기 때문에)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

#Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect=CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect=cnt_vect.transform(X_train)

In [11]:
# X_train으로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect=cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


>> 11314개의 문서에서 피처, 즉 단어가 101631개로 만들어짐

### 2-1. Count 기반으로 피처 데이터화된 데이터에 로지스틱 회귀를 적용해 분류 예측해보기

#피처 데이터화된 데이터에 로지스틱 회귀를 적용해 분류 예측해보기
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학/예/평 수행
lr_clf=LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred=lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

### 2-2. TF-IDF 기반으로 피처 데이터화된 데이터에 로지스틱 회귀를 적용해 분류 예측해보기

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect=TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학/예/평 수행
lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.674


TF-IDF가 Count 기반보다 훨씬 더 높은 예측 정확도를 제공 >> 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 TF-IDF 벡터화가 좋은 예측 결과를  도출함

### 2-3. TfidfVectorizer의 파라미터를 적용해보기(스톱워드 지정 등)

In [18]:
#stop_words 필터링 추가하고 ngram을 (1,1)>(1,2)로 변경해 피처 벡터화 적용
tfidf_vect=TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.692


### 2-4. GridSearchCV를 이용해 로지스틱 회귀 하이퍼 파라미터 최적화 수행하기

In [20]:
from sklearn.model_selection import GridSearchCV

#최적의 C 값 도출 튜닝 수행, cv는 3폴드 세트로 설정
params={'C':[0.01,0.1,1,5,10]}
grid_cv_lr=GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_)

#최적의 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred=grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.701


>> 로지스틱 회귀의 C가 10일때 G의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냄 + 전보다 성능 수치도 향상

## 3. 사이킷런 파이프라인 사용 및 GridSearchCV와의 결합

- 사이킷런의 Pipeline 클래스는 데이터의 전처리와 머신러닝 학습 과정을 통일된 API 기반에서 처리할 수 있음 >> 직관적인 ML 코드 생성 가능
  + 대용량의 데이터의 피처 벡터화 결과를 별도로 저장하지 않고 스트림 기반에서 바로 ML 데이터로 입력할 수 있음 >> 수행시간 절약
  + 피처 벡터화랑 데이터 전처리 작업과 Estimator를 결합할 수 있음

- TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression객체를 lr_clf로 생성한 뒤 두 객체를 파이프라인으로 연결
- TfidfVectorizer의 학습 데이터와 테스트 데이터에 대한 fit(), transform() 수행을 통한 피처 벡터화와 LogisticRegression의fit(),predict() 수행을 통한 머신러닝 모델의 학습과 예측이 Pipeline의 fit(),predict()로 통일되어 수행됨.

In [24]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression객체를 lr_clf로 생성하는 Pipeline 생성
pipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),('lr_clf',LogisticRegression(C=10))])

# 별도의 TfidfVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(),predict()가 필요없음
# pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측 가능
pipeline.fit(X_train,y_train)
pred=pipeline.predict(X_test)
print('Pipeline을 통한 LogisticRegression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

Pipeline을 통한 LogisticRegression의 예측 정확도는 0.701


- GridSearchCV에 Pipeline을 입력하면서 TfidfVectorizer 파라미터와 LogisticRegression의 하이퍼 파라미터를 함께 최적화
- GridSearchCV에 Estimator 대신 Pipeline을 입력할 경우 param_grid의 입력값 설정이 약간 다름 > Key값이 tfidf_vect_ngram_range와 같이 '하이퍼 파라미터명+객체의 변수명'으로 제공

In [ ]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),('lr_clf',LogisticRegression())])

# Pipeline에 기술된 각각의 객체 변수에 언더바 2개를 연달아 붙여 (하이퍼)파라미터 이름과 값을 설정
params={'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],'tfidf_vect__max_df':[100,300,700],'lr_clf__C':[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe=GridSearchCV(pipeline,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train,y_train)
print(grid_cv_pipe.best_params_,grid_cv_pipe.best_score_)

pred=grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 LogisticRegression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))